In [14]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy


data = pd.read_csv("data.csv")
print(data['content'].info())
dataFiltered = data["content"][:35000]

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
dataFiltered=dataFiltered.apply(remove_stopwords)

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=dataFiltered.tolist()
#print(text_list[5])
tokenized_reviews = lemmatization(text_list)
#print(tokenized_reviews[5])

# -- Omar was here --------------------------------------------------------------------------------

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100, chunksize=100, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))



<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalog

---------- 
 Topics
 [(0, '0.038*"flight" + 0.034*"m." + 0.033*"car" + 0.031*"water" + 0.027*"plane" + 0.023*"passenger" + 0.022*"airport" + 0.021*"p." + 0.021*"air" + 0.020*"fire"'), (1, '0.033*"military" + 0.032*"missile" + 0.026*"administration" + 0.025*"nuclear" + 0.022*"country" + 0.022*"korean" + 0.017*"policy" + 0.016*"weapon" + 0.016*"leader" + 0.015*"foreign"'), (2, '0.112*"woman" + 0.051*"black" + 0.048*"white" + 0.040*"man" + 0.026*"community" + 0.024*"group" + 0.023*"right" + 0.023*"Police" + 0.022*"people" + 0.020*"female"'), (3, '0.024*"company" + 0.020*"job" + 0.018*"money" + 0.018*"year" + 0.017*"business" + 0.017*"tax" + 0.013*"economic" + 0.012*"trade" + 0.011*"market" + 0.011*"worker"'), (4, '0.075*"police" + 0.049*"officer" + 0.022*"man" + 0.018*"protest" + 0.017*"gun" + 0.016*"incident" + 0.016*"protester" + 0.014*"people" + 0.013*"murder" + 0.013*"suspect"'), (5, '0.022*"book" + 0.015*"animal" + 0.014*"dog" + 0.011*"image" + 0.010*"item" + 0.009*"bag" + 0.009*"spa

In [15]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis


/home/shehab/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/home/shehab/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/shehab/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/shehab/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's do

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.039280  0.041936       1        1  17.320490
18     0.129563  0.011531       2        1  10.539241
14     0.120285 -0.091018       3        1   8.095798
3      0.150993  0.154253       4        1   7.058981
13     0.072708 -0.069478       5        1   5.859514
19    -0.148150 -0.080200       6        1   5.593117
16    -0.015784 -0.011176       7        1   5.220517
8      0.091999 -0.148155       8        1   5.037026
15     0.077117 -0.087023       9        1   4.441274
9      0.151447  0.048379      10        1   4.080954
4     -0.106611 -0.259131      11        1   4.051274
17     0.023825  0.161993      12        1   3.265269
12    -0.112570  0.005730      13        1   3.143515
7      0.040232  0.042884      14        1   3.098036
2     -0.151132  0.056791      15        1   2.759147
1      0.195075  0.001762      16        1   2.659325
6      0.027407  0.175993      17        1   2.411106
5     -0.283152  0.190319      18        1   2.194784
0     -0.127750 -0.215784      19        1   1.740082
10    -0.174782  0.070396      20        1   1.430551, topic_info=          Term          Freq         Total Category  logprob  loglift
696      woman  25468.000000  25468.000000  Default  30.0000  30.0000
503     police  18161.000000  18161.000000  Default  29.0000  29.0000
568     school  12888.000000  12888.000000  Default  28.0000  28.0000
1129    medium  17819.000000  17819.000000  Default  27.0000  27.0000
107     people  62027.000000  62027.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
2634  incident    640.134597   7015.590257  Topic20  -4.8973   1.8529
1558     study    593.168507   5629.152349  Topic20  -4.9735   1.9969
831      group    716.670523  21061.016864  Topic20  -4.7844   0.8665
3718    letter    505.727998   4250.501169  Topic20  -5.1330   2.1183
881     member    529.945011  17679.903246  Topic20  -5.0862   0.7397

[1310 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1879      2  0.053138    .
1879      4  0.340400    .
1879      5  0.101353    .
1879      7  0.187937    .
1879     14  0.316944    .
...     ...       ...  ...
704       7  0.019804   ’s
704       9  0.008694   ’s
704      10  0.031535   ’s
704      11  0.002346   ’s
704      14  0.004416   ’s

[3114 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 19, 15, 4, 14, 20, 17, 9, 16, 10, 5, 18, 13, 8, 3, 2, 7, 6, 1, 11])

In [16]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.632715345779237

Coherence Score:  0.5338857923819255
